In [ ]:
# What things can you do on your turn in Hanabi?
# 1. Play a card from your hand. -- Choose to play, choose a card.
# 2. Discard a card from your hand. -- Choose to discard, choose a card.
# 3. Give a hint to the other player about cards in their hand. -- Choose to give hint, choose a hint.

# What information do you have at any point in the Hanabi Game?
# 1. Hints given for each of your cards.
# 2. Number of remaining hints.
# 3. Number of cards left in the deck.
# 4. Number of lives left.
# 5. Hints given to other players already.
# 6. Cards in play.
# 7. Score.

# Maybe we can describe the state by [[score], [lives remaining], [hints available], [# cards in deck],
#                                     [Cards in play], [Hints in Hand], [Other People's Hands Hints]]

In [ ]:
# We will have a game manager script that takes in a list of players and sends them the game state when it is their turn.
# We then instantiate a couple of players and have them play games trying to achieve the highest score.

In [1]:
import itertools
import random

class HanabiGame:
    
    def __init__(self, hand_size = 5, hints = 8, lives = 3, num_players = 2, num_suits = 5, num_values = 5):
                    
        # Hand Size
        if hand_size < 1:
            print("How are you going to play without any cards in your hand, silly?")
        self.hand_size = hand_size
        
        # Hints remaining
        self.hints = hints
        self.max_hints = self.hints
        
        # Lives remaining
        self.lives = lives

        # Number of players
        if num_players < 2:
            print("Hanabi must be played with at least two players.")
            
        self.num_players = num_players
        
        # Suits of Cards
        self.num_suits = num_suits
        self.suits = list(range(self.num_suits))

        # Values of Cards
        # There are 3 zeros of each suit, one of the maximal value, and two of all others.
        if num_values < 2:
            print("Please allow for at least two values.")
        
        self.num_values = num_values
        self.values = [0, 0, 0] + list(range(1, self.num_values-1))*2 + [self.num_values-1]

        # Deck
        # A card in the deck's ten's place is the suit of the card and 
        # the one's place is the value
        self.deck = [(suit,value) for suit, value in itertools.product(self.suits,self.values)]

        # Cards in the Hands of each Player
        self.player_hands = [[] for i in range(num_players)]
        self.initialDeal()

        # Hint information available to each player
        # self.player_hints[0] = [Hints received for suits, Hints received for values, turns_in_hand]
        # Hints received are either 0 (no info), 1 (positive info), or -1 (negative info)
        # So, [[-1, 0, 0, 0, 0], [-1, 1, -1, -1, -1], 3] would correspond to knowing a card is
        # NOT of suit 0 and IS of value 1 and has been in your hand for 3 turns.
        self.player_hints = [[[[0]*self.num_suits,[0]*self.num_values, 0] for j in range(self.hand_size)]
                             for i in range(self.num_players)]
        
        # Keep track of whose turn it is
        self.current_player_turn = 0
        
        # Keep track of the score.
        # Note*: Normally in Hanabi your score is zero until the last turn
        # is played, but to help the AI, the current score will only become
        # zero when all lives are lost.
        self.current_score = 0
        
        # Cards that have been played onto the board
        # -1 corresponds to no card of a particular suit having been played.
        # The current value of self.board[i] = Highest value of card played
        # of that suit.
        self.board = [-1]*self.num_suits
        
        # Discard Pile
        # self.discards[i][j] = The number of suit i value j cards that have been discarded so far.
        self.discards = [[0]*num_values for i in range(self.num_suits)]
        
        # Keep a list of possible next moves
        self.legal_move_list = []
        # Initialize legal_move_list
        self.legalMoves()
        
        # Keep track of the number of moves left when the final round starts
        self.final_round_moves = self.num_players
        self.game_is_ending = False
        
        # When the game ends, the game manager should deal with cleaning everything up.
        self.game_end = False


    def initialDeal(self):
        random.shuffle(self.deck)
        for i in range(self.hand_size):
            for hand in range(self.num_players):
                self.player_hands[hand].append(self.deck.pop())
    
    # Returns the board state from a given player's perspective.
    # In particular, it returns all public information and the cards in the opponents hands.
    # [[score]: 1, [lives remaining]: 1, [hints available]: 1, [# cards in deck]: 1,
    # [Cards in play]: num_suits, [Discards]: num_suits*num_values, [Hints in Hand], [Other People's Hands Hints]]
    def boardState(self, player_number):
        return [self.current_score, self.lives, self.hints, len(self.deck), self.board, self.discards, 
                self.player_hints[player_number]] + [hint for i,hint in enumerate(self.player_hints) if i!= player_number]
    
    def printBoardState(self, player_number=-1):
        
        # If no player is specified, return the view from the current player
        if player_number == -1:
            player_number = self.current_player_turn
#         cur_board = boardState(player_number)
        print(f"From the perspective of player {player_number}")
        print(f"The current score is {self.current_score}.")
        print(f"You have {self.lives} lives and {self.hints} hints left.")
        print(f"There are {len(self.deck)} cards left in the deck.")
        for i, max_card in enumerate(self.board):
            if max_card == -1:
                print(f"You have not played any cards of suit {i}")
            elif max_card == 0:
                print(f"You have played the 0 of suit {i}.")
            else:
                print(f"You have played {max_card} cards of suit {i}.")
        
        for card_index, hint in enumerate(self.player_hints[player_number]):
            if hint == [[0]*self.num_suits, [0]*self.num_values, 0]:
                print(f"You don't know anything about card {card_index}.")
            else:
                if hint[0] == [0]*self.num_suits:
                    print(f"You don't know anything about the suit of card {card_index}.")
                else:
                    if 1 in hint[0]:
                        print(f"You know that {card_index} is of suit {(hint[0]).index(1)}")
                    else:
                        for suit_index, existence in enumerate(hint[0]):
                            if existence == -1:
                                print(f"You know card {card_index} is not of suit {suit_index}.")
                
                if hint[1] == [0]*self.num_values:
                    print(f"You don't know anything about the value of card {card_index}.")
                else:
                    if 1 in hint[1]:
                        print(f"You know that {card_index} is of suit {(hint[0]).index(1)}")
                    else:
                        for value_index, existence in enumerate(hint[1]):
                            if existence == -1:
                                print(f"You know card {card_index} is not of value {value_index}.")

                            
    # Returns a tuple (The index of the current player's turn, [List of allowable moves])    
    def legalMoves(self):
        
        # Moves will be described by a tuple 
        # (Choose to Play/Discard/Hint, Index of Card to Discard/Play [-1 if giving hint instead], 
        #                              Hint Type[(player, suit, value)])
        # Hint Type is (-1,-1,-1) if not giving a hint.
        allowed_moves = []
        
        # Play moves (0,X,-1,-1)
        for i in range(self.hand_size):
            allowed_moves.append((0,i,-1,-1,-1))
        
        # Discard moves (1,X,-1,-1)
        for i in range(self.hand_size):
            allowed_moves.append((1,i,-1,-1,-1))
        
        
        # If hint available, hint moves (2, -1, Player, Suit, Value)
        if self.hints > 0:
            for player_index in range(self.num_players):
                if player_index != self.current_player_turn:
                    for suit_index in range(self.num_suits):
                        allowed_moves.append((2,-1,player_index,suit_index,-1))
                    for value_index in range(self.num_values):
                        allowed_moves.append((2,-1,player_index,-1,value_index))
        
        self.legal_move_list = allowed_moves
        return (self.current_player_turn, allowed_moves)
    
    def printLegalMoves(self):
        self.legalMoves()
        print(f"It is Player {self.current_player_turn}'s turn.")
        if len(self.legal_move_list) == 0:
            print("You done goofed. No legal moves.")
            return
        for index, move in enumerate(self.legal_move_list):
            if move[0] == 0:
                print(f"{index}: You can play your {move[1]} card.")
            elif move[0] == 1:
                print(f"{index}: You can discard your {move[1]} card.")
            else:
                if move[3] != -1:
                    print(f"{index}: You can tell player {move[2]} about their cards of suit {move[3]}.")
                elif move[4] != -1:
                    print(f"{index}: You can tell player {move[2]} about their cards of value {move[4]}.")
                else:
                    print('We definitely should not have gotten here.')
    
    # Current Player Draws a card
    def drawCard(self):
        if len(self.deck) == 0:
            self.game_is_ending = True
            self.final_round_moves -= 1
            if self.final_round_moves == 0:
                self.endGame()
            if len(self.player_hints[self.current_player_turn]) < 5:
                self.player_hints[self.current_player_turn].append([[0]*self.num_suits,[0]*self.num_values, 0])
            return

        if len(self.player_hands[self.current_player_turn]) != 4:
            print(f"A player is trying to draw with {len(self.player_hands[self.current_player_turn])} cards in their hand.")
            return
        else:
            self.player_hands[self.current_player_turn].append(self.deck.pop())
            self.player_hints[self.current_player_turn].append([[0]*self.num_suits,[0]*self.num_values, 0])
        
        self.legalMoves()
    
    # Implement this
    def endGame(self):
        self.game_end = True
        return
    
    def playCard(self, index):
        if index > len(self.player_hands[self.current_player_turn]):
            print("That card is unplayable. Uh oh.")
            return
        
        card = self.player_hands[self.current_player_turn][index]
        suit = card[0]
        value = card[1]
        
        
        # If the card is playable, play it, increment score, and draw a card.
        # Otherwise, we'll lose a life, check game end, discard it, and draw a card.
        if self.board[suit] + 1 == value:
            self.board[suit] = value
            self.current_score += 1
        else:
            self.lives -= 1
            if self.lives <= 0:
                self.endGame()
                return
        self.discardCard(index, gain_hint = False)
        self.drawCard()

    # Only draw a card if gain_hint = True
    # Update discard pile
    def discardCard(self, index, gain_hint = True):
        self.discards[self.player_hands[self.current_player_turn][index][0]][
                            self.player_hands[self.current_player_turn][index][1]]+= 1
        del self.player_hands[self.current_player_turn][index]
        del self.player_hints[self.current_player_turn][index]
        if gain_hint == True:
            self.drawCard()
            if self.hints < self.max_hints:
                self.hints += 1
        return

    # If a hint is given whilst game is ending, remember to decrement final_round_moves
    # self.player_hints = [[[[0]*self.num_suits,[0]*self.num_values] for j in range(self.hand_size)]
    #                        for i in range(self.num_players)]
    def giveHint(self, hint):
        player, suit_hint, value_hint = hint[0], hint[1], hint[2]
        
        if self.hints <= 0:
            print("You cannot give a hint, as you have no hint tokens left!")
        
        # card_hints is of the form [[suit hints], [value hints]]
        for card_index, card_hints in enumerate(self.player_hints[player]):
            card_suit = self.player_hands[player][card_index][0]
            card_value = self.player_hands[player][card_index][1]
            if (suit_hint != -1):
                if suit_hint == card_suit:
                    card_hints[0] = [1 if x == card_suit else -1 for x in range(self.num_suits) ]
                else:
                    card_hints[0][suit_hint] = -1
            elif (value_hint != -1):
                if value_hint == card_value:
                    card_hints[1] = [1 if x == card_value else -1 for x in range(self.num_values) ]
                else:
                    card_hints[1][suit_hint] = -1
            else:
                print("We were given neither a suit nor a value.")
                return
        self.hints -= 1
        return

    
    def performMove(self, index = -1, move = None):
        self.legalMoves()
        if index == -1 and move == None:
            print("What move would you like to do?")
            return
        if index > len(self.legal_move_list):
            print("Choose a valid move. [Index out of range.]")
            return
        if move != None and move not in legal_move_list:
            print("Choose a valid move. [Move not legal.]")
            return

        if index != -1:
            move = self.legal_move_list[index]
        
#         print(move)
#         else:
#             index = legal_move_list.index(move)
        
        # Reminder: move = (play/discard/hint, card to play/discard index, player to give hint, suit hint, value hint)
        # Reminder: move[0]: 0 - Play, 1 - Discard, 2 - Hint
        if move[0] == 0:
            self.playCard(move[1])
        elif move[0] == 1:
            self.discardCard(move[1])
        elif move[0] == 2:
            self.giveHint((move[2], move[3], move[4]))
        
        # Next player's turn
        self.legalMoves()
        for card in self.player_hints[self.current_player_turn]:
            # Increment the time the card has been in the player's hand
            card[2] += 1
        self.current_player_turn = (self.current_player_turn + 1) % self.num_players
        

In [2]:
test_game = HanabiGame()

In [67]:
test_game.printBoardState()

From the perspective of player 0
The current score is 0.
You have 3 lives and 8 hints left.
There are 40 cards left in the deck.
You have not played any cards of suit 0
You have not played any cards of suit 1
You have not played any cards of suit 2
You have not played any cards of suit 3
You have not played any cards of suit 4
You don't know anything about card 0.
You don't know anything about card 1.
You don't know anything about card 2.
You don't know anything about card 3.
You don't know anything about card 4.


In [68]:
test_game.printLegalMoves()

It is Player 0's turn.
0: You can play your 0 card.
1: You can play your 1 card.
2: You can play your 2 card.
3: You can play your 3 card.
4: You can play your 4 card.
5: You can discard your 0 card.
6: You can discard your 1 card.
7: You can discard your 2 card.
8: You can discard your 3 card.
9: You can discard your 4 card.
10: You can tell player 1 about their cards of suit 0.
11: You can tell player 1 about their cards of suit 1.
12: You can tell player 1 about their cards of suit 2.
13: You can tell player 1 about their cards of suit 3.
14: You can tell player 1 about their cards of suit 4.
15: You can tell player 1 about their cards of value 0.
16: You can tell player 1 about their cards of value 1.
17: You can tell player 1 about their cards of value 2.
18: You can tell player 1 about their cards of value 3.
19: You can tell player 1 about their cards of value 4.


In [ ]:
test_game.performMove(index = 10)

In [ ]:
test_game.printLegalMoves()

In [ ]:
test_game.printBoardState()

In [ ]:
test_game.discards

In [ ]:
# Implementation of Game Manager that will run games for the player AI and provide training

In [20]:
import numpy as np
import random
import statistics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

class HanabiPlayer:
    
    # Game state will be ordered as
    # [[score]: 1, [lives remaining]: 1, [hints available]: 1, [# cards in deck]: 1,
    # [Cards in play]: num_suits, [Discards]: num_suits*num_values, [Hints in Hand], [Other People's Hands Hints]]
    def __init__(self, game_state_size, weights, num_players = 2):
        
        self.game_state_size = game_state_size
        self.num_players = 2
        
        # These are random. Just a guess!
        second_layer_size = 32
        third_layer_size = 16
        
        # Allows for modular layer sizes
        # The final layer is the number of possible moves to output.
        # There are always hand_size*2 (plays or discards) + num_opponents * (num_values + num_suits) (hints)
        # possible moves. FOR NOW -- We're gonna assume normal hand size with 5 suits and 5 values for convenience.
        # Hence, 5*2 + (num_players - 1)*10
        # I also put the game_state_size as the [-1] element since it 'comes before' 0. This is terrible. Why have
        # I chosen to do this?
        self.layer_sizes = [second_layer_size, third_layer_size, 10+10*(self.num_players-1), game_state_size]
        
        self.model = Sequential([
            # Input will be a vector of game_state_size length
            Dense(units=self.layer_sizes[0], input_shape=(game_state_size,), use_bias=False, activation='relu'),
            Dense(units=self.layer_sizes[1], use_bias=False, activation='relu'),
            Dense(units=self.layer_sizes[2], use_bias=False, activation='relu')
        ])
        
#         if any(weights == None):
#             self.weights = np.array([0]*((game_state_size)*64+64*16+16*20))
#         else:
        
        self.set_weights(weights)
        
        # Since we will be updating the weights ourselves, this isn't necessary...
        self.model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
    
    def get_weights(self):
        weights = []
        for i, layer in enumerate(self.model.layers):
            weights.extend(np.resize(layer.get_weights()[0], self.layer_sizes[i-1]*self.layer_sizes[i]).tolist())
#         print(weights)
        return weights
        
    # Set the given weights for the model
    def set_weights(self, np_flat_weights):
        if len(np_flat_weights) == (self.game_state_size*self.layer_sizes[0]+
                                    self.layer_sizes[0]*self.layer_sizes[1]+
                                    self.layer_sizes[1]*self.layer_sizes[2]):
            weights0 = np_flat_weights[:self.game_state_size*self.layer_sizes[0]].copy()
            weights1 = np_flat_weights[self.game_state_size*self.layer_sizes[0]: 
                                      self.game_state_size*self.layer_sizes[0]+
                                      self.layer_sizes[0]*self.layer_sizes[1]].copy()
            weights2 = np_flat_weights[-self.layer_sizes[1]*self.layer_sizes[2]:].copy()
            
#             print("final weight to be added: ", weights2[-1])
            
            weights0.resize(self.game_state_size,self.layer_sizes[0])
            weights1.resize(self.layer_sizes[0],self.layer_sizes[1])
            weights2.resize(self.layer_sizes[1],self.layer_sizes[2])

            self.model.layers[0].set_weights([weights0])
            self.model.layers[1].set_weights([weights1])
            self.model.layers[2].set_weights([weights2])
        else:
            print(f"game_state_size = {self.game_state_size}, layer sizes = {self.layer_sizes[0]}, {self.layer_sizes[1]}, {self.layer_sizes[2]}.")
            print(f"Incorrect weight sizes. We expected {(self.game_state_size*self.layer_sizes[0]+self.layer_sizes[0]*self.layer_sizes[1]+self.layer_sizes[1]*self.layer_sizes[2])} but got {len(np_flat_weights)}.")
            
            return
        
    
    def choose_best_move(self, game_state):
        guess_vec = self.model.predict(np.array( [game_state,] ))
        
#         print(f"guess vec = {guess_vec}")
        return guess_vec[0]
                
    def update_game(self, game_state):
        self.game_state = game_state
        return
    
    def mutate(self, dna_weights):
        mutation_rate = 0.001
        new_weights = []
        
        for weight in dna_weights:
            if random.random() < mutation_rate:
                new_weights.append(random.random())
            else:
                new_weights.append(weight)
        
        return new_weights

# test_game = HanabiGame()
# input_size = len(flatten(test_game.boardState(0)))+len(test_game.legal_move_list[0])
# print(input_size)
# test_player = HanabiPlayer(game_state_size = input_size)
# # len(board_state) = 4 + num_suits + num_suits*num_values + hand_size*(num_suits+num_values+1)*num_players + len(move)
# # In 2 player, 5 cards, 5 suits, 5 values, you should get 144
# weights = test_player.model.get_weights()
# test_player.model.summary()

In [21]:
import numpy as np
import random
import statistics

def create_offspring_weights(parents):
    all_weights = [list(parent.get_weights()) for parent in parents]
    average_weights = []
    for i in range(len(all_weights[0])):
        average_weights.append(sum([weight[i]/len(all_weights) for weight in all_weights]))
        
    return mutate(average_weights)

def mutate(weights, mutation_rate=0.01):
    new_weights = []

    for weight in weights:
        if random.random() < mutation_rate:
            new_weights.append(random.random())
        else:
            new_weights.append(weight)

    return np.array(new_weights)


In [24]:
def flatten(list_to_flatten):
        found_something_to_flatten = True
        while(found_something_to_flatten == True):
            found_something_to_flatten = False
            temp_list = []
            for i, item in enumerate(list_to_flatten):
                if isinstance(item, list):
                    found_something_to_flatten = True
                    temp_list += list_to_flatten[i]
                else:
                    temp_list.append(list_to_flatten[i])
            list_to_flatten = temp_list
        return list_to_flatten
    
test_game = HanabiGame()

flat_state = flatten(test_game.boardState(0))
input_size = len(flat_state) # This equals 144 in 2 player normal game
print(f"The input length is {input_size}.")

# Genetic Algorithm to find a good HanabiAI
# Initialize with 2*N players (Initial Population)

population = 2*50
# The weights in this current implementation are 5,440
num_weights = 5440

# current_generation_players = prev_gen
current_generation_players = []

for i in range(population):
    # [Player, score_achieved]
    random_weights = np.random.rand(num_weights)
    current_generation_players.append([HanabiPlayer(game_state_size=input_size,weights=random_weights), 0])

    
# Evalute the fitness of each player (which we'll consider to be the maximum score obtained before game ends.)

# I am going to implement it now where pairs of players play different games, but maybe it'd be better 
# to have all pairs of players play through the same game for fairness. If they happen to play an easy game
# They might get a bias.

#How many times will we train the group?
num_gen_to_train = 10000

for i in range(num_gen_to_train):
    # Sort randomly and take pairs in order as game partners
    random.shuffle(current_generation_players)
        
    for j in range(int(population/2)):
        cur_game = HanabiGame()
#         player_turn = 0
        while cur_game.game_end == False:
            # Predict a move
            np_bs = np.array(flatten(cur_game.boardState(cur_game.current_player_turn)))
            move_ratings = current_generation_players[2*j+cur_game.current_player_turn][0].choose_best_move(np_bs)
            move_index = 0
            
            
            # If we are out of hints, play/discard
            # This is a hack atm, where on the final round of the game, the players will only
            # play/discard and not give hints, which is not necessarily optimal.
            if cur_game.hints <= 0 or cur_game.game_is_ending:
                move_index = move_ratings[:10].argmax()
            else:
                move_index = move_ratings.argmax()
#             print(f"We are attempting to perform move {move_index} with {cur_game.hints} hints.")
#             print(move_index)
            cur_game.performMove(move_index)
        # Score the players this time
        current_generation_players[2*j][1]=cur_game.current_score
        current_generation_players[2*j+1][1]=cur_game.current_score
    
##    We want things that scored 0 to have the minimum chance of being score, otherwise
##     the more points the more likely we'd like them to be chosen so, (score+1)^2 seems
##     like a good bias.

#     We have  changed the fitness to throw out those who score zero... gl us!
    fitness_from_score = [(x[1])**2 for x in current_generation_players]
    normalized_fitness = np.array(fitness_from_score) / np.sum(fitness_from_score)
    
    print(f"Fitness = {normalized_fitness}")

    
    # Save weights after training? Or maybe save N of them?
    # print(current_generation_players[0])
    sorted_gen = sorted(current_generation_players, key=lambda x: x[1], reverse=True)
    zeros = 0
    while (sorted_gen[-(zeros+1)][1] == 0):
        zeros += 1
    print(f"In generation {i} there were {zeros} players who scored 0 points.\n")
    print(f"First place in generation {i} scored : {sorted_gen[0][1]}")
    print(f"Last place in generation {i} scored : {sorted_gen[-1][1]}")
    f = open("./weights/genetic_weights.txt", "a")
    f.write(f"Best of gen {i} scored {sorted_gen[0][1]} : \n" + str(sorted_gen[0][0].get_weights()) + "\n")
    f.close()

    
    # Generate a new generation by randomly choosing two* parents biased upon fitness
    # Create a child with new weights based upon the cross-over of parents
    
    new_generation = []
    for j in range(population):
        # Two random parents chosen with score bias
        index_choice = list(range(len(current_generation_players)))
        parents = [np.random.choice(index_choice, p=normalized_fitness),
                   np.random.choice(index_choice, p=normalized_fitness)]
        parents = [current_generation_players[choice][0] for choice in parents]
#         print(parents)
        child_weights = create_offspring_weights(parents)
#         print(flatten(child_weights))
        new_generation.append([HanabiPlayer(game_state_size=input_size,weights=child_weights),0])
    
    current_generation_players=new_generation
    
    # Mutate with some small chance
    # Repeat




# Save the previous generation to run it through again.
prev_gen = current_generation_players

The input length is 144.
Fitness = [0.02409639 0.02409639 0.         0.         0.05421687 0.05421687
 0.0060241  0.0060241  0.         0.         0.02409639 0.02409639
 0.02409639 0.02409639 0.         0.         0.05421687 0.05421687
 0.         0.         0.         0.         0.         0.
 0.0060241  0.0060241  0.         0.         0.02409639 0.02409639
 0.         0.         0.         0.         0.         0.
 0.0060241  0.0060241  0.         0.         0.02409639 0.02409639
 0.0060241  0.0060241  0.         0.         0.         0.
 0.         0.         0.         0.         0.02409639 0.02409639
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.0060241  0.0060241  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.0060241  0.0060241  0.         0.         0.05421687 0.05421687
 0.         0.         0.         0.         0.0060241  0.0060241
 0.         0.   

Fitness = [0.         0.         0.01639344 0.01639344 0.00409836 0.00409836
 0.         0.         0.00409836 0.00409836 0.01639344 0.01639344
 0.01639344 0.01639344 0.         0.         0.01639344 0.01639344
 0.00409836 0.00409836 0.03688525 0.03688525 0.00409836 0.00409836
 0.00409836 0.00409836 0.06557377 0.06557377 0.00409836 0.00409836
 0.00409836 0.00409836 0.03688525 0.03688525 0.01639344 0.01639344
 0.03688525 0.03688525 0.00409836 0.00409836 0.         0.
 0.01639344 0.01639344 0.01639344 0.01639344 0.         0.
 0.         0.         0.00409836 0.00409836 0.         0.
 0.         0.         0.00409836 0.00409836 0.00409836 0.00409836
 0.00409836 0.00409836 0.00409836 0.00409836 0.03688525 0.03688525
 0.01639344 0.01639344 0.00409836 0.00409836 0.01639344 0.01639344
 0.03688525 0.03688525 0.         0.         0.00409836 0.00409836
 0.01639344 0.01639344 0.         0.         0.         0.
 0.         0.         0.         0.         0.01639344 0.01639344
 0.         0.   

Fitness = [0.00326797 0.00326797 0.         0.         0.02941176 0.02941176
 0.0130719  0.0130719  0.0130719  0.0130719  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00326797 0.00326797 0.00326797 0.00326797 0.0130719  0.0130719
 0.0130719  0.0130719  0.05228758 0.05228758 0.00326797 0.00326797
 0.0130719  0.0130719  0.         0.         0.02941176 0.02941176
 0.02941176 0.02941176 0.         0.         0.         0.
 0.         0.         0.0130719  0.0130719  0.0130719  0.0130719
 0.02941176 0.02941176 0.0130719  0.0130719  0.02941176 0.02941176
 0.         0.         0.00326797 0.00326797 0.         0.
 0.         0.         0.0130719  0.0130719  0.00326797 0.00326797
 0.         0.         0.05228758 0.05228758 0.         0.
 0.00326797 0.00326797 0.00326797 0.00326797 0.0130719  0.0130719
 0.         0.         0.         0.         0.         0.
 0.00326797 0.00326797 0.08169935 0.08169935

Fitness = [0.01136364 0.01136364 0.02556818 0.02556818 0.01136364 0.01136364
 0.02556818 0.02556818 0.00284091 0.00284091 0.01136364 0.01136364
 0.         0.         0.00284091 0.00284091 0.01136364 0.01136364
 0.00284091 0.00284091 0.01136364 0.01136364 0.01136364 0.01136364
 0.00284091 0.00284091 0.01136364 0.01136364 0.04545455 0.04545455
 0.02556818 0.02556818 0.00284091 0.00284091 0.04545455 0.04545455
 0.00284091 0.00284091 0.         0.         0.         0.
 0.         0.         0.         0.         0.00284091 0.00284091
 0.02556818 0.02556818 0.02556818 0.02556818 0.00284091 0.00284091
 0.         0.         0.00284091 0.00284091 0.00284091 0.00284091
 0.00284091 0.00284091 0.         0.         0.04545455 0.04545455
 0.         0.         0.         0.         0.01136364 0.01136364
 0.         0.         0.01136364 0.01136364 0.02556818 0.02556818
 0.01136364 0.01136364 0.         0.         0.01136364 0.01136364
 0.00284091 0.00284091 0.02556818 0.02556818 0.01136364 0.01

Fitness = [0.         0.         0.00980392 0.00980392 0.02205882 0.02205882
 0.         0.         0.00245098 0.00245098 0.00245098 0.00245098
 0.         0.         0.03921569 0.03921569 0.         0.
 0.03921569 0.03921569 0.00980392 0.00980392 0.         0.
 0.         0.         0.03921569 0.03921569 0.         0.
 0.02205882 0.02205882 0.         0.         0.         0.
 0.         0.         0.02205882 0.02205882 0.         0.
 0.         0.         0.03921569 0.03921569 0.00980392 0.00980392
 0.         0.         0.         0.         0.00245098 0.00245098
 0.         0.         0.00980392 0.00980392 0.         0.
 0.         0.         0.03921569 0.03921569 0.         0.
 0.00245098 0.00245098 0.00980392 0.00980392 0.00245098 0.00245098
 0.00980392 0.00980392 0.00980392 0.00980392 0.00245098 0.00245098
 0.06127451 0.06127451 0.00980392 0.00980392 0.         0.
 0.         0.         0.         0.         0.00245098 0.00245098
 0.         0.         0.00980392 0.00980392 0.03

Fitness = [0.02122642 0.02122642 0.00943396 0.00943396 0.00235849 0.00235849
 0.00943396 0.00943396 0.         0.         0.00235849 0.00235849
 0.00943396 0.00943396 0.02122642 0.02122642 0.05896226 0.05896226
 0.         0.         0.03773585 0.03773585 0.         0.
 0.00235849 0.00235849 0.00943396 0.00943396 0.00235849 0.00235849
 0.         0.         0.05896226 0.05896226 0.         0.
 0.05896226 0.05896226 0.         0.         0.         0.
 0.00943396 0.00943396 0.         0.         0.02122642 0.02122642
 0.00943396 0.00943396 0.03773585 0.03773585 0.00235849 0.00235849
 0.         0.         0.         0.         0.         0.
 0.         0.         0.02122642 0.02122642 0.         0.
 0.02122642 0.02122642 0.         0.         0.00235849 0.00235849
 0.00235849 0.00235849 0.         0.         0.00235849 0.00235849
 0.00943396 0.00943396 0.00235849 0.00235849 0.00235849 0.00235849
 0.00943396 0.00943396 0.03773585 0.03773585 0.00235849 0.00235849
 0.         0.         0.

KeyboardInterrupt: 

In [ ]:
# Save the previous generation to run it through again.
prev_gen = current_generation_players

In [ ]:
f = open("./weights/genetic_weights.txt", "w")
f.write("gen 85 -- " + str(prev_gen[0][0].get_weights()) + "\n")
f.close()

f = open("./weights/genetic_weights.txt", "r")
print(f.read())

In [ ]:
flat_state = flatten(test_game.boardState(0))
input_size = len(flat_state) # This equals 144 in 2 player normal game
print(f"The input length is {input_size}.")

test_play1 = HanabiPlayer(game_state_size=input_size,weights=np.random.rand(10560))
test_play2 = HanabiPlayer(game_state_size=input_size,weights=np.random.rand(10560))

test_game1 = HanabiGame()


board_state_np = np.array(flatten(test_game1.boardState(0)))
print(len(board_state_np))
test_play1.model.predict(np.array( [board_state_np,] ))

In [ ]:
[1,2,3,4][-2:]

In [ ]:
A =[1,2,3]
B = [3]
B.append(A);B

In [ ]:
HanabiPlayer()

In [ ]:
A = [1,2,3,4]
A = np.resize(A, (2,2))

In [ ]:
np.resize(A,4)

In [ ]:
test_player.model.layers[0].get_weights()[0].shape

In [ ]:
test_player.model.layers[0].get_weights()[0]

In [ ]:
new_weights = np.random.rand(1,149)
new_bias = np.random.rand(149)
print(new_weights, new_bias)

In [ ]:
test_player.model.layers[0].set_weights([new_weights,new_bias])

In [ ]:
test_player.model.layers[0].get_weights()

In [ ]:
for i in range(len(test_player.model.layers)):
    print(i, test_player.model.layers[i].get_weights()[0].shape)

In [ ]:
[i for i in range(10) if i!=2]

In [ ]:
print(f"Is{' not' if True else ''}!")

In [ ]:
A = [[1,2],[3,4]]
for a in A:
    a[0] = 0
A

In [ ]:
A = [0,0,1]
B = ['a' if x == 0 else 'b' for x in A ]

In [ ]:
B

In [ ]:
random.random()

In [ ]:
flatten([[[1,2],[3,4]],[[5,6],[7,8]]])

In [ ]:
[1,2] + [1,2,3]

In [ ]:
A = [1]
A.append([[1,2],[3,4],[5,6]])
print(A)

In [ ]:
input_shape=(28,28,1)
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape,name='input'),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3),kernel_initializer=tf.keras.initializers.zeros(),
                               activation="relu",use_bias=True,name='conv1'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="relu",use_bias=True,bias_initializer='zeros',name='dense1'),
        tf.keras.layers.Dense(1, activation="softmax",name='output'),
    ]
)

In [ ]:
model.summary()

In [ ]:
np.resize(model.layers[0].get_weights()[0], 10)

In [ ]:
model = Sequential([
            # Input will be a vector of game_state_size length
            Dense(units=64, input_shape=(145,), use_bias=False, activation='relu'),
            Dense(units=16, use_bias=False , activation='relu'),
            Dense(units=20, use_bias=False , activation='sigmoid')
        ])

In [ ]:
model.layers[0].get_weights()

In [ ]:
np.random.rand(2,3)

In [ ]:
A = np.random.rand(2,3)
A.resize(6)
B = A[2:6]
B.resize(2,2);B

In [ ]:
game_a = HanabiGame()
game_b = game_a
game_b.boardState(0)

In [ ]:
game_b.performMove(index=0)
game_a.boardState(1)

In [ ]:
game_b.legal_move_list